In [ ]:
import anndata

rna_path = "data/multiomic_rna.h5ad"
methyl_path = "data/multiomic_methylation.h5ad"
data = {
    "rna": anndata.read_h5ad(rna_path),
    "methyl": anndata.read_h5ad(methyl_path)
}

In [ ]:
import numpy as np

data["rna"].X = data["rna"].X.astype(np.float32)
data["methyl"].X = data["methyl"].X.astype(np.float32)

We really ought to have a Bernoulli model for the methylation data. Should be straightforward to implement, but since right now the priority is testing the join operation, we'll settle for a normal model.

In [ ]:
from scdesigner.simulator import scdesigner
from scdesigner.margins.marginal import Normal

sims = {
    "rna": scdesigner(data["rna"], Normal("~ bs(UMAP1_integrated) * bs(UMAP2_integrated)"), max_epochs=3),
    "methyl": scdesigner(data["methyl"], Normal("~ bs(UMAP1_integrated) * bs(UMAP2_integrated)"), max_epochs=3)
}

Joining with a copula only makes sense for vertical integration (same samples, different features). This example doesn't have that, but we'll create a version of the data that mimics this pattern.

In [ ]:
from scdesigner.join import join_copula

sims["methyl"].anndata.obs = sims["rna"].anndata.obs.iloc[:len(sims["methyl"].anndata), :]
sim_joined = join_copula(sims["rna"], sims["methyl"], cov_fun = lambda x: np.cov(x))
sim_joined.sample()